<a href="https://colab.research.google.com/github/fevdaele/Machine_Learning_course_UGent_D012554_kaggle/blob/master/PredictionsFemke11/05/2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt;
import numpy as np;
import pandas as pd;
import seaborn as sns;
sns.set_context("notebook", font_scale=1.4);
sns.set_style("whitegrid");
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

### **Load train and dataset**

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

In [18]:
trainset.head(5)
#trainset.sort_values(by='label')


,AF3,F7,F3,FC5,T7,P7,O1,02,P8,T8,FC6,F4,F8,AF4,label
0,4299.49,3997.44,4277.95,4116.92,4353.85,4623.08,4100.00,4623.59,4202.56,4229.23,4211.79,4278.97,4600.00,4369.23,1
1,4302.05,3985.64,4261.03,4129.74,4334.36,4615.38,4072.31,4585.64,4192.31,4225.13,4195.90,4283.08,4607.18,4358.46,0
2,4321.03,4015.90,4265.13,4122.56,4333.33,4613.33,4072.82,4602.05,4192.31,4223.08,4155.38,4286.15,4608.21,4371.79,0
3,4408.21,4104.10,4380.00,4232.31,4449.74,4750.26,4169.23,4731.28,4311.28,4352.31,4319.49,4388.21,4715.90,4464.10,0
4,4347.18,3975.38,4266.67,4102.56,4333.33,4617.95,4097.44,4612.82,4210.77,4240.51,4248.21,4313.33,4664.10,4411.79,1


**Are there any missing values?**



In [19]:
trainset.isnull().sum().sum()

0

**How many datapoints are there for each class?**

In [20]:
print(trainset["label"].value_counts())

0    1099
1     901
Name: label, dtype: int64


### **Data pre processing**


In [21]:
data_train = trainset.copy()
data_train.pop('label')

data_test = testset.copy()
data_test.pop('index')

0            0
1            1
2            2
3            3
4            4
         ...  
12887    12887
12888    12888
12889    12889
12890    12890
12891    12891
Name: index, Length: 12892, dtype: int64

**Scaling of trainset**

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train = scaler.fit_transform(data_train)


#plt.figure(figsize=(12,8))
#trainset_scaled.boxplot(vert=False)
#plt.show()

**Scaling of testset**

In [0]:
data_test = scaler.transform(data_test)
#plt.figure(figsize=(12,8))
#testset_scaled.boxplot(vert=False)
#plt.show()


### ***Model selection***

***Define model***

In [24]:
# Since we have two class data sets, a logistic regression model will be selected to fit the train data.

model = LogisticRegression()
score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)



0.6593382417786088


### ***Model optimization***

In [25]:
#logistic regression models only have one hyperparameter, C. To optimize the model, the hyperparameter needs to be changed.
from sklearn.model_selection import GridSearchCV

model = LogisticRegression()
search_space = [0.001,0.01,0.1,1,10,100]
gridDict = dict(C=search_space)
params = gridDict
grid_search = GridSearchCV(model, param_grid=params, scoring = 'roc_auc', cv=10)
grid_search.fit(data_train, trainset.label)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
{'C': 100}
0.6596007906466621


In [26]:
model = LogisticRegression(C=100)
score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)

0.6596007906466621


### ***Fit model on trainset***

In [27]:
model.fit(data_train, trainset['label'])

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### ***Make predictions***

In [28]:
predictions = model.predict_proba(data_test)
print(predictions)

[[0.76329792 0.23670208]
 [0.64229297 0.35770703]
 [0.68605999 0.31394001]
 ...
 [0.52637569 0.47362431]
 [0.36303085 0.63696915]
 [0.57893101 0.42106899]]


In [32]:

PredictionsFemke = pd.DataFrame({'index': testset['index'], 'label': predictions[:,1]})



print(PredictionsFemke)

       index     label
0          0  0.236702
1          1  0.357707
2          2  0.313940
3          3  0.391177
4          4  0.833815
...      ...       ...
12887  12887  0.469027
12888  12888  0.560396
12889  12889  0.473624
12890  12890  0.636969
12891  12891  0.421069

[12892 rows x 2 columns]


In [0]:
filename = "PredictionsFemke.csv"

#make sure to not write the Pandas index column (index=False)
PredictionsFemke.to_csv(filename,index=False)